In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from glob import glob
import os
from datetime import datetime
import matplotlib
import matplotlib.patches as patches

matplotlib.rcParams.update({'errorbar.capsize': 4})

In [1]:
SETUP_NAME = "Barium-Tagging"
EXCITATION_FILTER = "Center-527nm_Width-22nm",
DOUBLE_FILTER = "Center-561nm_Width-21nm"
LEFTOVER_FILTER = "Center-500nm_Width-29nm",
LASER_WAVELENGTH = "515nm"
LASER_SOURCE = "TOptica"
DICHROIC_MIRROR_CUTTOFF = "550nm"
N_DMIRRORS = 3
MICROSCOPE_OBJECTIVE = "Mitutoyo_50x_NA_0.5"
exposure = 10
EXPOSURE_TIME = str(exposure)+'s'

def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

def write_settings_static(fout):

    settings = [SETUP_NAME, LASER_WAVELENGTH, 
                LASER_SOURCE, DICHROIC_MIRROR_CUTTOFF,
                N_DMIRRORS, MICROSCOPE_OBJECTIVE,
                EXPOSURE_TIME]
    # Make a list of strings with the name of the variable and its value:
    fsettings = [f'{namestr(v, globals())[0]}: {v}' for v in settings]  

    with open(fout, 'w') as fo:
        fo.write('\n'.join(fsettings))

def read_settings(fin):
    df_settings = pd.read_csv(fin, sep=':', names=['setting', 'value'], index_col='setting')
    return df_settings



In [2]:
DICT_MICROSCOPE_OBJECTIVE = {1: "Mitutoyo_50x_NA_0.5",
                             2: "Nikon_60x_NA_0.95"}

DICT_LASER_SOURCE = {1: "TOptica",
                     2: "Diode"}

def edit_dropdown(key, dictionary):
    dropdowns = {'laser': DICT_LASER_SOURCE, 'obj': DICT_MICROSCOPE_OBJECTIVE}
    current_dict = dropdowns[dictionary]
    while True:
        print(f'\nEditing {df_settings.setting[key]} = {df_settings.value[key]}')
        print(current_dict)
        print("\nPress 'q' to quit\n" )

        subkey = input('\nSelect option\n')
        if subkey == 'q':
            break
        elif int(subkey) in current_dict:
            subkey = int(subkey)
            df_settings.loc[subkey, 'value'] = current_dict[subkey]
            break
        else:
            print("invalid key")
            break

def edit_settings():
    while True:
        print(df_settings)
        print("\nPress 'q' to quit\n" )
        key = input('\nSelect key\n')
        if key == 'q':
            break
        elif int(key) in df_settings.index:
            key = int(key)
            if (df_settings.setting[key] == 'LASER_SOURCE'):
                edit_dropdown(key, 'laser')
            elif  (df_settings.setting[key] == 'MICROSCOPE_OBJECTIVE'):
                edit_dropdown(key, 'obj')
            else:
                print(f'\nEditing {df_settings.setting[key]} = {df_settings.value[key]}')
                value = input('\nEnter value:\n')
                df_settings.loc[key, 'value'] = value
        
        else:
            print("invalid key")
            break
        return df_settings

In [ ]:
fout = '/Users/pabloherrero/Library/CloudStorage/GoogleDrive-qmlab@mail.huji.ac.il/My Drive/Ba Tagging/data/img/sets/08-12-24/Rhodamine-B 50 µM/1/settings.txt'

write_settings_static(fout)

In [ ]:
edit_settings()

In [ ]:
pathfind = '/Users/pabloherrero/Library/CloudStorage/GoogleDrive-qmlab@mail.huji.ac.il/My Drive/Ba Tagging/data/img/sets/'
def find_recent_settings(pathfind):
    finds = glob(pathfind+'**/**/**/settings.txt')
    dirs = [f[:len(pathfind)+8] for f in finds]
    datestr = [os.path.split(d)[1] for d in dirs]
    dates = [datetime.strptime(dstr, "%d-%m-%y") for dstr in datestr]
    dir_recent = datetime.strftime(max(dates), "%d-%m-%y")
    path_recent = os.path.join(pathfind, dir_recent)
    recent_settings_path = glob(path_recent+'/**/**/settings.txt')[-1]
    return recent_settings_path

In [ ]:
recent_settings_path = find_recent_settings(pathfind)
read_settings(recent_settings_path)